```{=typst}
#set text(
  font: ("Times New Roman", "LXGW WenKai"),
  size: 11pt,
)

不妨设第 $i$ 个点的坐标为 $(x_i, y_i)$，同时第一个点的坐标为 $(0, 0)$，那么问题转化为使下列条件成立的点的集合：

$
min sum_(i j)[(x_i - x_j)^2 + (y_i - y_j)^2 - d_(i j)]^2
$

#set math.mat(delim: "[")
$
记 quad x = mat(x_2, x_3, ..., x_25; y_2, y_3, ..., y_25) quad "扁平化处理便于计算"
$

调用 `scipy.optimize.leastsq` 函数进行求解，得到最优解 $x$，然后将 $x$ 还原为 $(x_i, y_i)$ 的形式即可。

```


In [ ]:
import numpy as np
import scipy.optimize as opt

In [ ]:
N = 52
XN = 25

data = [
  [4, 1, 0.9607],
  [12, 1, 0.4399],
  [13, 1, 0.8143],
  [17, 1, 1.3765],
  [21, 1, 1.2722],
  [5, 2, 0.5294],
  [16, 2, 0.6144],
  [17, 2, 0.3766],
  [25, 2, 0.6893],
  [5, 3, 0.9488],
  [20, 3, 0.8000],
  [21, 3, 1.1090],
  [24, 3, 1.1432],
  [5, 4, 0.4758],
  [12, 4, 1.3402],
  [24, 4, 0.7006],
  [8, 6, 0.4945],
  [13, 6, 1.0559],
  [19, 6, 0.6810],
  [25, 6, 0.3587],
  [8, 7, 0.3351],
  [14, 7, 0.2878],
  [16, 7, 1.1346],
  [20, 7, 0.3870],
  [21, 7, 0.7511],
  [14, 8, 0.4439],
  [18, 8, 0.8363],
  [13, 9, 0.3208],
  [15, 9, 0.1574],
  [22, 9, 1.2736],
  [11, 10, 0.5781],
  [13, 10, 0.9254],
  [19, 10, 0.6401],
  [20, 10, 0.2467],
  [22, 10, 0.4727],
  [18, 11, 1.3840],
  [25, 11, 0.4366],
  [15, 12, 1.0307],
  [17, 12, 1.3904],
  [15, 13, 0.5725],
  [19, 13, 0.7660],
  [15, 14, 0.4394],
  [16, 14, 1.0952],
  [20, 16, 1.0422],
  [23, 16, 1.8255],
  [18, 17, 1.4325],
  [19, 17, 1.0851],
  [20, 19, 0.4995],
  [23, 19, 1.2277],
  [24, 19, 1.1271],
  [23, 21, 0.7060],
  [23, 22, 0.8052],
]


def distance(x1, x2, y1, y2):
  return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)


def total_distance(x):
  x[0] = 0
  x[XN] = 0
  res = np.zeros(N)
  for index, item in enumerate(data):
    p1, p2, d = item
    res[index] = (distance(x[p1 - 1], x[p2 - 1], x[p1 + XN - 1], x[p2 + XN - 1]) - d) ** 2
  return res


x0_1 = np.zeros(2 * XN)
X_1, cov = opt.leastsq(total_distance, x0_1)
print(X_1)

x0_2 = np.ones(2 * XN)
X_2, cov = opt.leastsq(total_distance, x0_2)
print(X_2)

x0_3 = np.random.rand(2 * XN)
X_3, cov = opt.leastsq(total_distance, x0_3)
print(X_3)

x0_4 = np.random.rand(2 * XN)
X_4, cov = opt.leastsq(total_distance, x0_4)
print(X_4)

= 结论

从结果输出可知最优解与初值密切相关，并存在不收敛的现象，因此需要多次尝试不同的初值。

在上面的随机采样中，下面一组没达到最大迭代次数，可认为是收敛的：

```python
x0_3 = np.random.rand(N)
X_3, cov = opt.leastsq(total_distance, x0_3)
print(X_3)
```

对应的结果为：

```python
[ 0.          0.37765714  0.76553459 -0.55643979 -0.08783977  0.8954588
  0.46715544  0.37692024 -0.04734326  0.79567171  0.83367524  0.46253785
  0.27799172  0.18635986 -0.23417541  0.86068053  0.23013212 -0.44146135
  1.06084275  0.82623884  1.23492281  1.18623873  0.982989   -0.02264462
  0.64521199  0.          0.94748932  1.09746102  0.77333728  0.68935958
 -0.01310255  0.28364851 -0.00955153  0.67262237  0.05662742  0.65174302
 -0.07450919  0.80140373  0.42314099  0.63993537  1.3173152   1.32550543
  0.07466709  0.67331145  0.2732619   0.119839    0.27643256 -0.52554867
  0.2962078   0.28070909]
```

换算成 $(x_i, y_i)$ 的形式：

```python
[(0.0, 0.0), (0.37765714, 0.94748932), (0.76553459, 1.09746102), (-0.55643979, 0.77333728), (-0.08783977, 0.68935958), (0.8954588, -0.01310255), (0.46715544, 0.28364851), (0.37692024, -0.00955153), (-0.04734326, 0.67262237), (0.79567171, 0.05662742), (0.83367524, 0.65174302), (0.46253785, -0.07450919), (0.27799172, 0.80140373), (0.18635986, 0.42314099), (-0.23417541, 0.63993537), (0.86068053, 1.3173152), (0.23013212, 1.32550543), (-0.44146135, 0.07466709), (1.06084275, 0.67331145), (0.82623884, 0.2732619), (1.23492281, 0.119839), (1.18623873, 0.27643256), (0.982989, -0.52554867), (-0.02264462, 0.2962078), (0.64521199, 0.28070909)]
```
